In [31]:
import configparser
def parse_ini(path):
    config = configparser.ConfigParser(strict=False, allow_no_value=True, inline_comment_prefixes=(';',))
    config.read(path)
    data = {}
    for section in config.sections():
        data[section] = {}
        for key, value in config.items(section):
            if key in ['coders', 'gfx']: value = [v.strip() for v in value.split(',')]
            data[section][key] = value
    return data

import pathlib
ships = {}
for p in pathlib.Path('.').rglob('*.ini'):
    ships[p.stem] = parse_ini(p)

In [32]:
for ship in ships:
    if pathlib.Path(f'{ship}.txt').exists():
      with open(f'{ship}.txt') as f:
          ships[ship]['description'] = f.read().strip()
import json
with open('ships.json', 'w') as f:
    json.dump(ships, f, indent=2)

In [30]:
import zipfile
from PIL import Image
for ship in ships:
  cockpit = []
  panels = []
  with zipfile.ZipFile(f'{ship}.zip') as z:
    for f in sorted(z.namelist()):
      if f.lower().endswith('.bmp'):
        with z.open(f) as bmp:
          i = Image.open(bmp)
          if i.size == (55, 30):
            i.load()
            cockpit.append(i)
          elif i.size == (64, 100):
            i.load()
            panels.append(i)
    if cockpit:
      cockpit[0].save(f'{ship}-cockpit.png')
      cockpit[0].save(f'{ship}-cockpit.gif', save_all=True, append_images=cockpit[1:], optimize=True, duration=100, loop=0)
    assert panels, ship
    panels[0].save(f'{ship}-panel.png', save_all=True, append_images=panels[1:], optimize=True, duration=1000, loop=0)
